In [87]:
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
import numpy as np
from IPython.display import display

# Utility Functions

# Make the directory to hold the combined images
path_to_new_images = '/content/new_images/'
!mkdir -p {path_to_new_images}


def make_df(csv_name) -> pd.DataFrame:
    df = pd.read_csv(csv_name)
    df.columns = ["url1", "url2", "match-score"]
    return df



def get_images(img1_url, img_url2):
    try:
      response1 = requests.get(img1_url)
      try:
          response2 = requests.get(img_url2)
          img1 = Image.open(BytesIO(response1.content))
          img2 = Image.open(BytesIO(response2.content))
      except:
          print(f"error: could NOT display image on URL {row.url2}")
          return False, img1, None
    except:
        print(f"error: could not display image on URL {row.url1}")
        return False, None, None

    return True, img1, img2



def make_combo(img1, img2, idx, img_path):
    #Transform images to the same height, if necessary
    if img1.size[1] != img2.size[1]:
        img1 = img1.resize((img1.size[0], img2.size[1]))

    # Create a new image combining both
    new_img = Image.new('RGB', (img1.size[0] + img2.size[0], img1.size[1]))
    new_img.paste(img1, (0, 0))
    new_img.paste(img2, (img1.size[0], 0))

    # Save new image
    new_img.save(f"{img_path}/test_sample{str(idx).zfill(4)}.jpg")

    return new_img

In [ ]:
debug = False
test_size = 2


# import data into a dataframe
url_label_df = make_df('/content/pairwise_training_data.csv')
if debug: print(url_label_df.head()); example_size = test_size # number of images to create
else: example_size = len(url_label_df) # number of images to create


# Look at the URLs for small sample review
for i,(idx,row) in enumerate(url_label_df.iterrows()):
  if i < example_size:
      # print(f"{i}: {row.url1} vs {row.url2}")
      both, img1, img2 = get_images(row.url1, row.url2 )
      if not both: continue
      new_image = make_combo(img1, img2, idx, path_to_new_images)
      if debug: display(new_image)
